<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/ML_IB_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ibapi
!pip install yfinance
from ibapi.order import Order

import threading
import time  # Add this import at the beginning of your script
from ibapi.contract import Contract
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
import yfinance as yf
import pandas as pd
import datetime
import time
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Define stock tickers and date range
stocks = ['AAPL', 'GOOG', 'TSLA', 'ASRT', 'PBF']
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2022, 3, 7)

# Download stock data from Yahoo Finance
print("Downloading stock data...")
stock_data = yf.download(stocks, start=start_date, end=end_date)
print("Stock data downloaded successfully.")

# Create a new CSV file and write the data to it
stock_data.to_csv('stock_data.csv', index=True)

# Load the data
data = pd.read_csv("stock_data.csv")

# Drop columns that are not needed
stock_data = stock_data.drop(columns=['Adj Close'])
print(stock_data)

# Check if there are any missing values in the stock data
if stock_data.isnull().values.any():
    # Replace missing values with the column mean
    stock_data = stock_data.fillna(stock_data.mean())

# Normalize the data
stock_data_norm = (stock_data - stock_data.mean()) / stock_data.std()
print("Stock Norm:", stock_data_norm)

# Define the training, validation, and testing sets
train_data = stock_data_norm.iloc[:15]
val_data = stock_data_norm.iloc[0:15]
test_data = stock_data_norm.iloc[15:]

print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)

# Make sure that the training data contains at least two rows
if train_data.shape[0] < 2:
    raise ValueError("Training data must contain at least two rows.")

# Define the features and target variables for the testing set
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

# Reset the index of X_test and y_test
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Train a linear regression model on the training set
lr = LinearRegression()
lr.fit(X_test, y_test)

# Use the trained model to make predictions on the testing set
y_test_pred = lr.predict(X_test)

# Evaluate the performance of the model using the mean squared error
test_mse = mean_squared_error(y_test, y_test_pred)
print("Testing MSE:", test_mse)

# Define take profit and stop loss percentages
take_profit_percent = 0.03
stop_loss_percent = 0.01

# Display the list of best stocks to buy and sell
buy_codes = []
sell_codes = []

for i in range(len(stocks)):
    if y_test_pred[i] > stock_data_norm.iloc[-1, i]:
        buy_codes.append(stocks[i])
    else:
        sell_codes.append(stocks[i])

print("Buy Codes:", buy_codes)
print("Sell Codes:", sell_codes)


class IBTrader(EWrapper, EClient):
    # Add the following instance variable to the class
    def __init__(self, host, port, client_id):
        EWrapper.__init__(self)
        EClient.__init__(self, wrapper=self)

        self.host = host
        self.port = port
        self.client_id = client_id

        self.ticker_map = {}
        self.connect_event = threading.Event()

    # Add this method to the class
    def connect_and_run(self):
        self.connect(self.host, self.port, self.client_id)
        self.connect_event.wait()
        if not self.isConnected():
            print("Failed to connect to TWS or IB Gateway. Please check your connection settings.")
            return

        # Run the message loop
        thread = threading.Thread(target=self.run)
        thread.start()

    def nextValidId(self, order_id: int):
        super().nextValidId(order_id)
        self.connect_event.set()

def get_current_price(app, symbol):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"

    # Use a unique request ID based on the current timestamp
    req_id = int(time.time())

    # Request current market data for the specified contract
    app.reqMarketDataType(4)
    app.reqMktData(req_id, contract, "", False, False, [])

    # Sleep for a short period to allow time for the market data to be received
    time.sleep(5)

    # Get the current market data from the ticker
    ticker = app.ticker_map.get(req_id)
    if ticker is None:
        print(f"Failed to get market data for {symbol}.")
        return None

    current_price = ticker.last
    app.cancelMktData(req_id)

    return current_price



def create_take_profit_order(action, quantity, take_profit_price):
    order = Order()
    order.action = action
    order.orderType = "LMT"
    order.totalQuantity = quantity
    order.lmtPrice = take_profit_price
    order.tif = "GTC"  # Good-Til-Cancelled
    order.outsideRth = True  # Allow trading outside regular trading hours
    return order

##########################
def filter_stocks_by_pe_ratio(stocks, max_pe_ratio):
    filtered_stocks = []
    for code in stocks:
        pe_ratio = get_pe_ratio(code)
        if pe_ratio is not None and pe_ratio <= max_pe_ratio:
          # current_price = get_current_price(code)
            current_price = get_current_price(app, code)

            if current_price is not None:
                filtered_stocks.append((code, current_price))
    return filtered_stocks

##########################
def create_take_profit_order(action, quantity, take_profit_price):
    order = Order()
    order.action = action
    order.orderType = "LMT"
    order.totalQuantity = quantity
    order.lmtPrice = take_profit_price
    order.tif = "GTC"  # Good-Til-Cancelled
    order.outsideRth = True  # Allow trading outside regular trading hours
    return order

def take_profit_order(contract, order_id, num_shares, take_profit_price):
    order = Order()
    order.action = "SELL"
    order.orderType = "LMT"
    order.totalQuantity = num_shares
    order.lmtPrice = take_profit_price
    order.parentId = order_id
    order.transmit = True
    return order

def take_profit_order(contract, order_id, num_shares, take_profit_price):
    order = Order()
    order.action = "SELL"
    order.orderType = "LMT"
    order.totalQuantity = num_shares
    order.lmtPrice = take_profit_price
    order.parentId = order_id
    order.transmit = True
    return order

def get_pe_ratio(symbol):
    stock_info = yf.Ticker(symbol)
    pe_ratio = stock_info.info.get('trailingPE', None)
    return pe_ratio

# Define a function to filter the stocks based on their P/E ratio
def filter_stocks_by_pe_ratio(stocks, max_pe_ratio):
    filtered_stocks = []
    for code in stocks:
        pe_ratio = get_pe_ratio(code)
        if pe_ratio is not None and pe_ratio <= max_pe_ratio:
            filtered_stocks.append(code)
    return filtered_stocks

# Set the maximum P/E ratio for filtering the stocks
max_pe_ratio = 20

# Filter the stocks based on their P/E ratio
filtered_stocks = filter_stocks_by_pe_ratio(stocks, max_pe_ratio)

# Submit orders for buying or selling the stocks
buy_codes = []
sell_codes = []

for i in range(len(stocks)):
    if y_test_pred[i] > stock_data_norm.iloc[-1, i]:
        buy_codes.append(stocks[i])
    else:
        sell_codes.append(stocks[i])

# Risk Management
# Set a maximum amount of capital to be allocated to each trade
max_capital_per_trade = 10000

# Sell the stocks
for code in sell_codes:
    # Get the current stock price
    current_price = get_current_price(code)
    
    # Calculate the number of shares to sell based on the maximum capital per trade
    num_shares = int(max_capital_per_trade / current_price)
    
    # Calculate the stop-loss price as a percentage of the current price
    stop_loss_percent = 0.95  # set the stop-loss percentage to 5%
    stop_loss_price = current_price * stop_loss_percent
    
    #Place the sell order
    IBTrader.place_order(code, num_shares, "SELL")
    
    # Place a stop-loss order to limit potential losses
    stop_loss_order = Order()
    stop_loss_order.action = "SELL"
    stop_loss_order.orderType = "STP"
    stop_loss_order.totalQuantity = num_shares
    stop_loss_order.auxPrice = stop_loss_price
    
    IBTrader.placeOrder(IBTrader.order_id, Contract, take_profit_order)
    IBTrader.placeOrder(IBTrader.order_id, Contract, stop_loss_order)
    # Print the details of the order
    print(f"Sell order placed for {num_shares} shares of {code} at {current_price:.2f}")
    print(f"Take profit order placed for {num_shares} shares of {code} at {take_profit_price:.2f}")
    print(f"Stop loss order placed for {num_shares} shares of {code} at {stop_loss_price:.2f}")

# Buy the stocks
for code in buy_codes:
    # Get the current stock price
#    current_price = get_current_price(code)
    current_price = get_current_price(app, code)
    if current_price is not None:
    # Calculate the number of shares to buy based on the maximum capital per trade
      num_shares = int(max_capital_per_trade / current_price)

    # Calculate the take-profit and stop-loss prices as a percentage of the current price
    # ... (rest of the code)
    else:
       print(f"Failed to get the current price for {code}. Skipping calculations.")

    # Calculate the number of shares to buy based on the maximum capital per trade
    num_shares = int(max_capital_per_trade / current_price)
    
    # Calculate the take-profit and stop-loss prices as a percentage of the current price
    take_profit_percent = 1.05  # set the take-profit percentage to 5%
#    take_profit_price = current_price * take_profit_percent
    
    take_profit_price = current_price * (1 + take_profit_percent)
    take_profit_order = create_take_profit_order("SELL", num_shares, take_profit_price)


    stop_loss_percent = 0.95  # set the stop-loss percentage to 5%
    stop_loss_price = current_price * stop_loss_percent
    
    # Place the buy order
    app = IBTrader()
    app.connect("127.0.0.1", 4002, 0)

    IBTrader.place_order(code, num_shares, "BUY")
    app.placeOrder(IBTrader.order_id, Contract, take_profit_order)
    IBTrader.order_id += 1  # Increment the order ID for the next order

    # Place a take-profit order and a stop-loss order to limit potential gains and losses
    take_profit_order = Order()
    take_profit_order.action = "SELL"
    take_profit_order.orderType = "LMT"
    take_profit_order.totalQuantity = num_shares
    take_profit_order.lmtPrice = take_profit_price
    
    stop_loss_order = Order()
    stop_loss_order.action = "SELL"
    stop_loss_order.orderType = "STP"
    stop_loss_order.totalQuantity = num_shares
    stop_loss_order.auxPrice = stop_loss_price
    
    IBTrader.placeOrder(IBTrader.order_id, Contract, take_profit_order)
    IBTrader.placeOrder(IBTrader.order_id, Contract, stop_loss_order)
    
    # Print the details of the order
    print(f"Buy order placed for {num_shares} shares of {code} at {current_price:.2f}")
    print(f"Take profit order placed for {num_shares} shares of {code} at {take_profit_price:.2f}")
    print(f"Stop loss order placed for {num_shares} shares of {code} at {stop_loss_price:.2f}")

# Disconnect from the trading platform
IBTrader.disconnect()


